In [68]:
import pandas as pd

In [69]:
# A list containing the AOIs considered in the study
AOIS = ['Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4',
       'Paragraph5', 'Paragraph6', 'Text_Area', 'SubFigure1', 'SubFigure2',
       'SubFigure3', 'Figure', 'Formula']

In [70]:
# Read gaze data with events and AOIs using pandas library
data = pd.read_csv("data/gazeDataWithAOIsAndEvents.csv")
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [73]:
# Preview data
data.head()

,Row,Timestamp,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,128.9448,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,137.2724,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,500.0,567.9048,16.1935,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,145.6203,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,566.2654,4.7572,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,2.6143,-1369.9611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,153.9470,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,567.7859,27.6866,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,2753.7102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,162.2703,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,509.0,567.7856,27.6456,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,-4.9222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
# Drop gazes with GazeX or GazeY equal to NaN. This because these gazes are not mapped to any AOI
data = data.dropna(subset=['GazeX','GazeY'])

# Derive a dataframe with fixation data only

# Keep only relevant colomns
fixationData = data[['FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion']+AOIS].copy(deep=True)

# Map fixations to AOIs
"""
Goal: 
- Map fixations to AOIs
Context: 
- A fixation comprises several gazes.
- Following the NoteBook "1. Gaze Projections and AOIs Mapping", each gaze point is mapped to an AOI.
Problem: 
- Gazes belonging to the same fixation might not necessarily be mapped to the same AOI.
Solution: 
- When mapping fixations to AOIs, choose the AOI which is mapped to the majority of the gazes composing the fixation
"""                                
fixationData = fixationData.groupby(['FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion']).agg({aoi: (lambda x: pd.Series.mode(x)[0]) for aoi in AOIS }) 


# Flatten fixationData (i.e., a hierarchically indexed pandas.DataFrame)
fixationData.columns= [x for x in list(fixationData.columns.get_level_values(0))]
fixationData = fixationData.reset_index()

In [168]:
# Preview fixation data
fixationData.head()

,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
0,1.0,952.8727,495.6481,133.1086,358.0820,224.9734,0.2948,0,0,0,1,0,0,1,0,0,0,0,0
1,2.0,460.2509,170.6456,566.4076,708.1011,141.6935,0.1180,1,0,0,0,0,0,1,0,0,0,0,0
2,3.0,214.1108,166.5022,783.0360,891.3651,108.3291,0.1850,0,0,0,0,0,0,0,0,0,0,0,0
3,4.0,210.4464,198.8214,908.0385,1141.3419,233.3034,0.2571,1,0,0,0,0,0,1,0,0,0,0,0
4,5.0,289.7661,190.3990,1182.9961,1349.6581,166.6620,0.1125,1,0,0,0,0,0,1,0,0,0,0,0


In [169]:
# 1. AOI fixation measures

In [185]:
allStats = None

# Metrics for all AOIs aggregated

# Fixation duration aggregations
stats = fixationData.agg({'Fixation Duration':['sum','mean','max','min'], 'FixID':['count']}).unstack().to_frame().dropna().T

# edit stats formatting (by flattening it) to enable concating with the upcoming mesures
stats.columns = ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]

stats.insert(0, 'AOI', "All_AOIs")

allStats= stats

In [186]:
# Preview the metrics for all AOIs aggregated
display(allStats)

,AOI,Fixation Duration_max,Fixation Duration_mean,Fixation Duration_min,Fixation Duration_sum,FixID_count
0,All_AOIs,1249.8835,211.938128,66.5842,121652.4855,574.0


In [187]:
# Metrics for individual AOIs

# Iterate through the different AOIS
for aoi in AOIS:
    # For each AOI compute 'sum','mean','max','min' of 'Fixation Duration' 
    stats = fixationData[fixationData[aoi]==1].groupby(aoi).agg({'Fixation Duration':['sum','mean','max','min'],'FixID':['count']})
    
    # Flatten dataframe
    stats.columns= ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]
    
    # rename column aoi to "AOI" and set its value to aoi     
    stats.rename(columns={aoi:"AOI"}, inplace=True)
    stats["AOI"] = aoi
    
    #concate
    allStats = pd.concat([allStats, stats], axis=0)
    
# reset_index for allStats
allStats = allStats.reset_index(drop=True)

In [188]:
# Preview all stats
allStats

,AOI,Fixation Duration_max,Fixation Duration_mean,Fixation Duration_min,Fixation Duration_sum,FixID_count
0,All_AOIs,1249.8835,211.938128,66.5842,121652.4855,574.0
1,Paragraph1,574.8980,234.164400,66.6901,15923.1792,68.0
2,Paragraph2,183.3260,140.460357,66.6589,983.2225,7.0
3,Paragraph3,449.9495,253.757464,83.3021,5582.6642,22.0
4,Paragraph4,441.6774,155.939343,66.6725,1091.5754,7.0
5,Paragraph5,983.2385,176.806554,66.6587,7249.0687,41.0
6,Paragraph6,416.6124,161.567709,66.6328,3716.0573,23.0
7,Text_Area,983.2385,203.608859,66.6328,35020.7237,172.0
8,SubFigure1,916.5141,216.204068,66.6282,16647.7132,77.0
9,SubFigure2,1124.8921,215.932089,66.6423,32821.6776,152.0


In [ ]:
# 2. AOI visit measures

In [87]:
# documentation for the aprroach used to group columns by consecutive same value https://towardsdatascience.com/pandas-dataframe-group-by-consecutive-same-values-128913875dba

allStats = None

# Iterate through the different AOIS
for aoi in AOIS:
    
    #  Group by consecutive same value (CSAV) (see "exmaple of CSAV" for groups derived using AOI "Text_Area")
    csavGroups = fixationData.groupby((fixationData[aoi].shift() != fixationData[aoi]).cumsum())
    
    # Keep only groups where the AOI is activited (aoiName==1) 
    csavGroups = {k: v for k, v in groups if v[aoi].unique()==[1]}
    
    
    
    #  Compute visit aggregations
    #aggregations = groups.agg({'Fixation Duration':['sum','mean','max','min']})
    
#     for key in groups:
#         display(groups[key])


In [189]:
# "exmaple of CSAV" for groups derived using AOI "Text_Area"
for k, v in fixationData.groupby((fixationData['Text_Area'].shift() != fixationData['Text_Area']).cumsum()):
    print(f'[group {k}]')
    display(v)

[group 1]


,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
0,1.0,952.8727,495.6481,133.1086,358.0820,224.9734,0.2948,0,0,0,1,0,0,1,0,0,0,0,0
1,2.0,460.2509,170.6456,566.4076,708.1011,141.6935,0.1180,1,0,0,0,0,0,1,0,0,0,0,0


[group 2]


,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
2,3.0,214.1108,166.5022,783.036,891.3651,108.3291,0.185,0,0,0,0,0,0,0,0,0,0,0,0


[group 3]


...

,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
428,429.0,1393.6466,232.0286,461595.0564,461786.7234,191.6670,0.2105,0,0,0,0,0,0,0,0,0,0,0,1
429,430.0,1305.8203,205.4351,461803.3559,461961.6667,158.3107,0.2551,0,0,0,0,0,0,0,0,0,0,0,1
430,431.0,1579.1958,227.7136,462086.6497,462253.2897,166.6400,0.3733,0,0,0,0,0,0,0,0,0,0,0,1
431,432.0,1383.1400,273.0600,462728.2520,462936.5494,208.2974,0.2374,0,0,0,0,0,0,0,0,0,0,0,1
432,433.0,1320.5347,223.5956,462961.5472,463411.5315,449.9843,0.2688,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,546.0,1284.2194,382.6332,503890.3691,504098.6816,208.3125,0.1603,0,0,0,0,0,0,0,1,0,0,1,0
546,547.0,1502.0459,400.9053,504432.0191,504715.3627,283.3436,0.2150,0,0,0,0,0,0,0,0,1,0,1,0
547,548.0,1692.9673,400.8961,505515.1883,505765.1737,249.9854,0.1913,0,0,0,0,0,0,0,0,0,1,1,0
548,549.0,1722.4976,474.8149,505798.4850,506098.4692,299.9843,0.2847,0,0,0,0,0,0,0,0,0,1,1,0


[group 39]


,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
550,551.0,692.6540,259.4450,506356.7591,506423.4180,66.6589,0.2500,0,1,0,0,0,0,1,0,0,0,0,0
551,552.0,661.9447,208.9460,506540.1000,506673.4062,133.3062,0.1545,0,1,0,0,0,0,1,0,0,0,0,0
552,553.0,257.7143,231.3570,506798.3794,506965.0425,166.6631,0.2213,1,0,0,0,0,0,1,0,0,0,0,0
553,554.0,659.7778,467.7778,508095.3084,508170.2909,74.9826,0.2076,0,0,0,1,0,0,1,0,0,0,0,0
554,555.0,579.3611,178.5833,508386.9241,508536.9228,149.9987,0.1899,0,1,0,0,0,0,1,0,0,0,0,0
555,556.0,213.5194,201.9365,508620.2529,508861.9060,241.6531,0.2861,1,0,0,0,0,0,1,0,0,0,0,0
556,557.0,298.3518,176.2522,508903.5659,509020.2090,116.6431,0.2248,1,0,0,0,0,0,1,0,0,0,0,0
557,558.0,312.0404,693.2659,509111.8549,509236.8431,124.9882,0.2602,0,0,0,0,1,0,1,0,0,0,0,0
558,559.0,168.3077,840.5744,509378.4962,509611.8252,233.3290,0.3436,0,0,0,0,0,1,1,0,0,0,0,0
559,560.0,236.9259,820.0744,509661.8508,509853.4492,191.5983,0.2444,0,0,0,0,0,1,1,0,0,0,0,0


[group 40]


,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
568,569.0,1275.5329,469.3859,512069.8490,512311.4909,241.6419,0.3457,0,0,0,0,0,0,0,1,0,0,1,0
569,570.0,1422.3558,432.4423,512344.8194,512644.8059,299.9866,0.2192,0,0,0,0,0,0,0,0,1,0,1,0
570,571.0,1539.7246,413.0032,512669.8077,512869.7900,199.9822,0.2301,0,0,0,0,0,0,0,0,1,0,1,0
571,572.0,1668.5437,419.9558,512903.1218,513103.0798,199.9580,0.2457,0,0,0,0,0,0,0,0,0,1,1,0
572,573.0,1496.5625,306.5784,513353.0393,513586.3778,233.3385,0.2250,0,0,0,0,0,0,0,0,0,0,0,1
573,574.0,1223.7782,632.4227,516852.7076,517769.2217,916.5141,0.2980,0,0,0,0,0,0,0,1,0,0,1,0
